# Выполнение геометрических преобразований изображений

### Импорт необходимых библиотек

In [1]:
import sys

sys.path.insert(1, "../")

import os
import shutil
import pathlib
import collections

import cv2
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

from src.data.image_transformation import (
    read_points_for_transform,
    make_image_transformation,
    choose_transformation_matrix,
    get_transformation_matrix,
    perform_projective_transformation,
)
from src.data.utils import is_non_zero_file, get_directory_file_list
from src.features.finding_face_area import get_coords_of_face_area, get_face_area

### Поворот на 360 градусов изображений, снятых на тепловизор

In [2]:
main_path = pathlib.Path(
    pathlib.Path.home(),
    "Projects",
    "ml",
    "datasets",
    "FACE_ANTISPOOFING_EXP2",
)

In [3]:
src_path = main_path / "thermal" / "orig"
dst_path = main_path / "thermal" / "processed"

In [3]:
for root, dirs, files in os.walk(src_path):
    for filename in files:
        if not filename.startswith("."):
            image = Image.open(os.path.join(root, filename))
            image = image.rotate(360)
            image.save(dst_path / filename)

### Получение и сохранение матриц для проективного преобразования тепловых изображений

In [5]:
data_for_transform_path = main_path / "data_for_transform"

In [6]:
points_dir_path = data_for_transform_path / "points"

#### Генерация файлов для записи точек

In [8]:
rgb_images_dir_path = main_path / "rgb" / "orig"

In [9]:
th_images_dir_path = main_path / "thermal" / "processed"

In [7]:
for root, dirs, files in os.walk(rgb_images_dir_path):
    for filename in files:
        if not filename.startswith("."):
            open(points_dir_path / filename.replace("a.jpg", "points.txt"), "a").close()

#### Получение и сохранение матриц преобразований

In [10]:
matrices_dir_path = data_for_transform_path / "transformation_matrices"

In [9]:
for root, dirs, files in os.walk(points_dir_path):
    for filename in files:
        if not filename.startswith("."):
            fullpath_to_file = os.path.join(root, filename)
            if is_non_zero_file(fullpath_to_file):
                points_src, points_dst = read_points_for_transform(fullpath_to_file)
                M, status = cv2.findHomography(points_src, points_dst)

                with open(
                    matrices_dir_path / filename.replace("points.txt", "matrix.npy"),
                    "wb",
                ) as f:
                    np.save(f, M)

#### Вспомогательный скрипт для разметки (выделение пар точек) изображений

In [10]:
cont_points_filepaths = collections.OrderedDict()

In [11]:
def open_image_local(path_img):
    Image.open(path_img).show()


def open_txt_in_sublime(path_txt):
    import subprocess

    cmd = f"open -a /Applications/Sublime\ Text.app {path_txt}"
    p = subprocess.Popen(cmd, shell=True)
    p.wait()


def open_images_and_pointsfile(
    target_points_filename, rgb_images_dir_path, th_images_dir_path, points_dir_path
):
    open_image_local(
        rgb_images_dir_path / target_points_filename.replace("points.txt", "a.jpg")
    )
    open_image_local(
        th_images_dir_path / target_points_filename.replace("points.txt", "b.jpg")
    )
    open_txt_in_sublime(points_dir_path / target_points_filename)

In [12]:
for root, dirs, files in os.walk(points_dir_path):
    for filename in files:
        if not filename.startswith("."):
            cont_points_filepaths[filename] = None

cont_points_filepaths = collections.OrderedDict(sorted(cont_points_filepaths.items()))

In [13]:
with open(
    str(data_for_transform_path / "markup_state.json"),
    "r",
) as fp:
    cont_points_filepaths = json.load(fp)

In [14]:
for filename in cont_points_filepaths:
    if cont_points_filepaths[filename] is None:

        print("Начало обработки файла: ", filename)

        open_images_and_pointsfile(
            filename, rgb_images_dir_path, th_images_dir_path, points_dir_path
        )
        a = int(input())
        cont_points_filepaths[filename] = a

        with open(
            str(data_for_transform_path / "markup_state.json"),
            "w",
        ) as fp:
            json.dump(cont_points_filepaths, fp)

        # 1 — точки промаркированы
        # 0 — пропуск
        # 2 — временный пропуск
        # 9 — код остановки

        print("Обработка завершена, код: ", a)
        print("---------------------------")

        if a == 9:
            break

#### Применение матриц для преобразований и сохранение в директорию для валидации

In [14]:
validation_dir_path = main_path / "validation"

In [16]:
for root, dirs, files in os.walk(rgb_images_dir_path):
    for filename in files:
        if not filename.startswith("."):
            exp_id = filename.replace("_a.jpg", "")

            print("Обработка: ", exp_id)

            th_filename = exp_id + "_b.jpg"
            rgb_img = cv2.cvtColor(
                cv2.imread(str(rgb_images_dir_path / filename)), cv2.COLOR_BGR2RGB
            )
            th_img = cv2.imread(
                str(th_images_dir_path / th_filename), cv2.IMREAD_GRAYSCALE
            )
            matrix, matrix_file = get_transformation_matrix(exp_id, matrices_dir_path)
            th_img_res = perform_projective_transformation(th_img, rgb_img, matrix)
            coords_face = get_coords_of_face_area(
                cv2.cvtColor(rgb_img, cv2.COLOR_BGR2GRAY)
            )
            face_rgb = get_face_area(rgb_img, coords_face)
            face_th = get_face_area(th_img_res, coords_face)

            print("Успешно!")
            print()
            print("---------------------------")

            fig = plt.figure(figsize=(10, 7))
            fig.patch.set_facecolor("white")

            rows = 2
            columns = 2

            fig.add_subplot(rows, columns, 1)

            fig.suptitle(f"Experiment: {exp_id} (used matrix: {matrix_file})")

            plt.imshow(rgb_img)
            plt.axis("off")
            plt.title("RGB")

            fig.add_subplot(rows, columns, 2)

            plt.imshow(th_img_res, cmap="gray")
            plt.axis("off")
            plt.title("Thermal")

            fig.add_subplot(rows, columns, 3)

            # showing image
            plt.imshow(face_rgb)
            plt.axis("off")
            plt.title("RGB Face")

            fig.add_subplot(rows, columns, 4)

            plt.imshow(face_th, cmap="gray")
            plt.axis("off")
            plt.title("Thermal Face")

            plt.savefig(str(validation_dir_path / f"{exp_id}.png"))

#### Получение идентификаторов пар, не прошедших валидацию

In [13]:
validation_failed_dir_path = validation_dir_path / "failed"

In [18]:
failed_pairs = get_directory_file_list(validation_failed_dir_path)
failed_pairs_points = [f.replace(".png", "_points.txt") for f in failed_pairs]

In [19]:
len(failed_pairs_points)

In [20]:
with open(
    str(data_for_transform_path / "markup_state.json"),
    "r",
) as fp:
    cont_points_filepaths = json.load(fp)

In [21]:
for filename in failed_pairs_points:
    if cont_points_filepaths[filename] is None:

        print("Начало обработки файла: ", filename)

        open_images_and_pointsfile(
            filename, rgb_images_dir_path, th_images_dir_path, points_dir_path
        )
        a = int(input())
        cont_points_filepaths[filename] = a

        with open(
            str(data_for_transform_path / "markup_state.json"),
            "w",
        ) as fp:
            json.dump(cont_points_filepaths, fp)

        # 1 — точки промаркированы
        # 0 — пропуск
        # 2 — временный пропуск
        # 9 — код остановки

        print("Обработка завершена, код: ", a)
        print("---------------------------")

        if a == 9:
            break

#### Копируем наиболее подходящие матрицы преобразований для пар, у которых они не заданы и которые прошли валидацию

In [11]:
copy_matrices_dir_path = main_path / "copy_matrices"

In [23]:
for root, dirs, files in os.walk(points_dir_path):
    for filename in files:
        if not filename.startswith("."):
            fullpath_to_file = os.path.join(root, filename)
            exp_id = filename.replace("_points.txt", "")
            target_matrix_file = filename.replace("points.txt", "matrix.npy")

            if not os.path.exists(str(matrices_dir_path / target_matrix_file)):
                matrix_file = choose_transformation_matrix(exp_id, matrices_dir_path)
                shutil.copyfile(
                    str(matrices_dir_path / matrix_file),
                    str(copy_matrices_dir_path / target_matrix_file),
                )

                print(exp_id)
                print(matrix_file)
                print("---")

#### Скрипт для удаления матриц пар, для которых не размечены точки

In [24]:
for root, dirs, files in os.walk(points_dir_path):
    for filename in files:
        if not filename.startswith("."):
            fullpath_to_points_file = os.path.join(root, filename)
            target_matrix_file = filename.replace("points.txt", "matrix.npy")

            if not is_non_zero_file(fullpath_to_points_file):
                if os.path.exists(str(matrices_dir_path / target_matrix_file)):
                    os.remove(str(matrices_dir_path / target_matrix_file))
                    print("Файл удален!")

#### Применяем матрицы преобразований к тепловым изображениям и сохраняем результат

In [25]:
for root, dirs, files in os.walk(rgb_images_dir_path):
    for filename in files:
        if not filename.startswith("."):
            exp_id = filename.replace("_a.jpg", "")

            print("Обработка: ", exp_id)

            th_filename = exp_id + "_b.jpg"
            rgb_img = cv2.cvtColor(
                cv2.imread(str(rgb_images_dir_path / filename)), cv2.COLOR_BGR2RGB
            )
            th_img = cv2.imread(
                str(th_images_dir_path / th_filename), cv2.IMREAD_GRAYSCALE
            )
            matrix, matrix_file = get_transformation_matrix(exp_id, matrices_dir_path)
            th_img_res = perform_projective_transformation(th_img, rgb_img, matrix)
            cv2.imwrite(str(th_images_dir_path / th_filename), th_img_res)

            print("Успешно!")

#### Получаем и сохраняем лица в отдельные директории

In [27]:
# failed_fd = ["exp22_f_a_3a_x_e_a.jpg", "exp22_f_c_2a_x_e_a.jpg", "exp22_f_c_2b_x_e_a.jpg"]

In [12]:
rgb_face_images_dir_path = main_path / "rgb" / "faces"
thermal_face_images_dir_path = main_path / "thermal" / "faces"

In [15]:
for root, dirs, files in os.walk(rgb_images_dir_path):
    for filename in files:
        if not filename.startswith("."):
            exp_id = filename.replace("_a.jpg", "")

            print("Обработка: ", exp_id)

            th_filename = exp_id + "_b.jpg"
            rgb_img = cv2.cvtColor(
                cv2.imread(str(rgb_images_dir_path / filename)), cv2.COLOR_BGR2RGB
            )
            th_img = cv2.imread(
                str(th_images_dir_path / th_filename), cv2.IMREAD_GRAYSCALE
            )
            coords_face = get_coords_of_face_area(
                cv2.cvtColor(rgb_img, cv2.COLOR_BGR2GRAY)
            )
            face_rgb = get_face_area(rgb_img, coords_face)
            face_rgb = cv2.cvtColor(face_rgb, cv2.COLOR_BGR2RGB)
            face_th = get_face_area(th_img, coords_face)

            cv2.imwrite(str(rgb_face_images_dir_path / filename), face_rgb)
            cv2.imwrite(str(thermal_face_images_dir_path / th_filename), face_th)

            print("Успешно!")
            print("---------------------------")
            print()